In [1]:
import keras
from keras.layers import TFSMLayer
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

2025-06-10 00:01:41.002623: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-10 00:01:41.472627: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749513701.630962   46259 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749513701.674126   46259 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749513702.129605   46259 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [13]:
SEQ_LEN = 2
MODEL_PATH = '/src/airflow/model/varnn_macro_model'
CSV_PATH = '/src/data/future/VARNN/macro_future_09_06_2025.csv'

In [14]:
layer = TFSMLayer(MODEL_PATH, call_endpoint="serving_default")

In [15]:
df = pd.read_csv(CSV_PATH)
df = df.sort_values('date')  
features = df.drop(columns='date').columns
df_features = df[features]

In [16]:
scalers = {}
scaled_df = pd.DataFrame()
for col in features:
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaled_df[col] = scaler.fit_transform(df_features[[col]]).flatten()
    scalers[col] = scaler

In [17]:
diff_df = scaled_df.diff().fillna(0).reset_index(drop=True)

In [18]:
input_seq = diff_df.iloc[-SEQ_LEN:].values
input_tensor = np.expand_dims(input_seq, axis=0).astype(np.float32)

In [19]:
output_dict = layer(input_tensor)

In [20]:
predicted_diff = list(output_dict.values())[0].numpy()[0]

In [21]:
last_scaled = scaled_df.iloc[-1].values
undiff_scaled = predicted_diff + last_scaled

In [22]:
unscaled = []
for i, col in enumerate(features):
    val = scalers[col].inverse_transform([[undiff_scaled[i]]])[0, 0]
    unscaled.append(val)

In [23]:
print("Predicted next real-world values (real):")
for name, val in zip(features, unscaled):
    print(f"{name}: {val:.2f}")

Predicted next real-world values (real):
gold: 3350.48
oil: 63.38
us_dollar: 98.71
usd_vnd: 26054.46
cpi: 319.62
inflation_rate: -0.10
interest_rate: 4.33
